# Projet 3 - Analyse de données météo issues d’API

Ce projet vise à introduire au requêtage d'API et à la manipulation de données qui en sont issues, en utilisant des fonctions afin de faciliter la reproductibilité des analyses.

In [4]:
import requests

In [1]:
def renvoie_lat_long(localisation):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": localisation,
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        latitude = float(json_data[0]["lat"])
        longitude = float(json_data[0]["lon"])
        return (latitude, longitude)
    else:
        print(f"Erreur lors de la récupération des coordonnées de {localisation}")
        return None

In [6]:
LOCALISATION = "Paris"
lat, long = renvoie_lat_long(LOCALISATION)

print(f"Localisation : {LOCALISATION}, latitude : {lat}, longitude : {long}")

Localisation : Paris, latitude : 48.8534951, longitude : 2.3483915


In [8]:
def fetch_meteo_data(latitude, longitude):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&daily=temperature_2m_max,temperature_2m_min,humidity_2m_mean,wind_speed_10m_max&timezone=Europe/Paris"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return None

In [9]:
data = fetch_meteo_data(lat, long)
data

Erreur lors de la requête : 400 Client Error: Bad Request for url: https://api.open-meteo.com/v1/forecast?latitude=48.8534951&longitude=2.3483915&daily=temperature_2m_max,temperature_2m_min,humidity_2m_mean,wind_speed_10m_max&timezone=Europe/Paris


In [3]:
data = fetch_meteo_data(*renvoie_lat_long("Paris"))

# Afficher les clés principales
print(data.keys())

# Afficher les premières dates
print(data['time']['daily'][:5])

# Calculer la moyenne des températures maximales et minimales pour la première semaine
max_temps = data['daily']['temperature_2m_max'][:7]
min_temps = data['daily']['temperature_2m_min'][:7]

avg_max = sum(max_temps) / len(max_temps)
avg_min = sum(min_temps) / len(min_temps)

print(f"Température maximale moyenne pour la première semaine : {avg_max:.2f}°C")
print(f"Température minimale moyenne pour la première semaine : {avg_min:.2f}°C")

In [4]:
humidity = data['daily']['humidity_2m_mean']
wind_speed = data['daily']['wind_speed_10m_max']

# Calcul de l'indice de mauvaise coiffure
bad_hair_index = [hum * wind for hum, wind in zip(humidity, wind_speed)]

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# Paramètres pour Seaborn
sns.set_theme(style="darkgrid")

# Initialisation de la figure
plt.figure(figsize=(14, 6))

# Dessiner les courbes avec Seaborn
sns.lineplot(data['time']['daily'], data['daily']['temperature_2m_max'], label='Température maximale', color='red')
sns.lineplot(data['time']['daily'], data['daily']['temperature_2m_min'], label='Température minimale', color='blue')

# Titre et labels
plt.xlabel('Date')
plt.ylabel('Température (°C)')
plt.title("Évolution des températures sur une année à Paris")

# Affichage du graphique
plt.show()

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# Paramètres pour Seaborn
sns.set_theme(style="darkgrid")

# Initialisation de la figure
plt.figure(figsize=(14, 6))

# Dessiner la courbe avec Seaborn
sns.lineplot(x=data['time']['daily'], y=bad_hair_index, label='Indice de Mauvaise Coiffure', color='purple')

# Titre et labels
plt.xlabel('Date')
plt.ylabel('Indice')
plt.title("Indice de Mauvaise Coiffure sur une année à Paris")

# Affichage du graphique
plt.show()

In [7]:
import seaborn as sns
import numpy as np

def plot_heatmap(data):
    temperatures = np.array(data['daily']['temperature_2m_max']).reshape(-1, 7)
    days = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    
    plt.figure(figsize=(10, 30))
    sns.heatmap(temperatures, cmap='coolwarm', cbar_kws={'label': 'Température (°C)'}, xticklabels=days, yticklabels=False)
    plt.title('Heatmap des températures maximales par semaine à Paris')
    plt.xlabel('Jour de la semaine')
    plt.ylabel('Semaine de l\'année')
    plt.show()

plot_heatmap(data)